# Cats and dogs clasificator

Details

* Binary Classification problem
* Colored Images
* No Standard Dimension
* Kaggle Dataset > https://drive.google.com/open?id=16AlwTDOeyFaiP3RPxKOk5s80IycK80X4
* We will use subset of this data 



Data

Train (2000 images)
*   class_a (1000 images)
*   class_b (1000 images)
---
Valid
*   class_a (500 images)
*   class_b (500 images)
---
Test
*   Test_folder (1000 images)


In [ ]:
# libraries
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [ ]:
# training directories
train_dir = "/content/drive/MyDrive/Colab Notebooks/cats_and_dogs_small/train"
validation_dir = "/content/drive/MyDrive/Colab Notebooks/cats_and_dogs_small/validation"
test_dir = "/content/drive/MyDrive/Colab Notebooks/cats_and_dogs_small/test"

** Data preprocessing **
* Read the picture files.
* Decode the JPEG content to RGB grids of pixels.
* Convert these into floating point tensors.
* Rescale the pixel vales (between 0 and 255) to the [0, 1] interval.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
Link : https://keras.io/preprocessing/image/

In [ ]:
# Generating batches of tensor image data
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))

model.add(layers.MaxPooling2D((2 ,2))) # After this step we will have images of 75 x 75

model.add(layers.MaxPooling2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))

model.add(layers.MaxPooling2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))

model.add(layers.MaxPooling2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))

model.add(layers.Flatten())

model.add(layers.Dense(512, activation='relu'))

model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=50
)

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

In [ ]:
model.save("model.h5")

In [ ]:
from tensorflow.keras import backend as k

k.clear_session()
del model